# Prepare UAI account for Azure Content Safety enabled Llama 2 model deployment
  

### 1. Prerequisites

#### 1.1 Assign variables

In [ ]:
uai_name = f"chatpdfuami"

#### 1.2 Get credential

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

#### 1.3 Configure workspace 

In [ ]:
from azure.ai.ml import MLClient

try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace_name = "<AML_WORKSPACE_NAME>"

    # get a handle to the workspace
    ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)


subscription_id = ml_client.subscription_id
resource_group = ml_client.resource_group_name
workspace_name = ml_client.workspace_name
workspace_resource = ml_client.workspaces.get(workspace_name)
workspace_location = workspace_resource.location

print(f"Connected to workspace {workspace_name}")

### 2. Create a Managed Identity for the Azure AI Content Safety enabled Llama 2 endpoint

##### 2.1 Get a handle to the ManagedServiceIdentityClient

In [ ]:
from azure.mgmt.msi import ManagedServiceIdentityClient
from azure.mgmt.msi.models import Identity

msi_client = ManagedServiceIdentityClient(
    subscription_id=subscription_id,
    credential=credential,
)

##### 2.2 Create the User Assigned Identity:

In [ ]:
msi_client.user_assigned_identities.create_or_update(
    resource_group_name=resource_group,
    resource_name=uai_name,
    parameters=Identity(location=workspace_location),
)

##### 2.3 Retrieve the identity object

In [ ]:
uai_identity = msi_client.user_assigned_identities.get(
    resource_group_name=resource_group,
    resource_name=uai_name,
)
uai_principal_id = uai_identity.principal_id
uai_client_id = uai_identity.client_id
uai_id = uai_identity.id
print(f"UAI principal id: {uai_principal_id}")
print(f"UAI id: {uai_id}")